In [5]:
from itertools import product
from collections import Counter
import random
import requests
import pandas as pd
import time
import json
# from tqdm import tqdm
from tqdm.auto import tqdm
from getpass import getpass

In [ ]:
scenario_data = json.load(open("eng_scenarios_50k.json"))
NUMBER_OF_SCENARIOS = 5000 # len(scenario_data)

## Yandex GPT запросы

In [ ]:
CATALOG_ID = getpass("Yandex Cloud Catalog ID")
API_KEY = getpass("Yandex Cloud Service Account API Key")

### Send requests (async)

In [ ]:
def send_yandex_gpt_request(system_content, user_content):
  prompt = {
    "modelUri": f"gpt://{CATALOG_ID}/yandexgpt/latest",
    "completionOptions": {
        "stream": True,
        "temperature": 0,
        "maxTokens": "2000"
    },
    "messages": [
        {"role": "system", "text": system_content},
        {"role": "user", "text": user_content}
          ]
    }

  # url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
  url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completionAsync"
  headers = {
      "Content-Type": "application/json",
      "Authorization": f"Api-Key {API_KEY}"
  }

  response = requests.post(url, headers=headers, json=prompt)
  # print(response.json())
  return response.json().get("id")

In [ ]:
scenario_list = []

In [ ]:
for i in tqdm(range(NUMBER_OF_SCENARIOS)):

  system_content = scenario_data[i]["system_content"]
  user_content = scenario_data[i]["user_content"]
  scenario_info = scenario_data[i]["scenario_info"]

  try:
    req_id = send_yandex_gpt_request(system_content, user_content)

    scenario_info["req_id"] = req_id
    scenario_list.append(scenario_info)
  except:
    print("Problem!")
    continue
  # time.sleep(0.2)

df = pd.DataFrame(scenario_list)
df.to_pickle(f"requests_yandex_gpt_{len(scenario_list)}.pickle")

  0%|          | 0/5000 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Get responses

In [ ]:
def get_yandex_gpt_response(req_id, sleep_delay=2, log=False):
  result_url = f"https://llm.api.cloud.yandex.net/operations/{req_id}"
  headers = {
      "Authorization": f"Api-Key {API_KEY}"
  }

  for i in range(3): ## Number of Attempts
      result = requests.get(result_url, headers=headers)
      if log:
        print(result.json())

      if not result.json()["done"]:
        time.sleep(sleep_delay)
      else:
        return result.json()["response"]["alternatives"][0]["message"]["text"]

In [ ]:
for i in tqdm(range(len(scenario_list))):
  if scenario_list[i].get("result") is not None:
    continue

  scenario_list[i]["response"] = get_yandex_gpt_response(scenario_list[i]["req_id"], log=True)

df = pd.DataFrame(scenario_list)
df.to_pickle(f"results_yandex_gpt_{len(scenario_list)}.pickle")

In [ ]:
df["response"].isna().sum()